In [1]:
import torch

import pathpyG as pp

print('Running on', pp.config['torch']['device'])

Running on cuda


Let's create a simple path data set with four paths of length two:

- 2 x A -> C -> D  
- 2 x B -> C -> E 

Using the following index mappings

0: A  
1: B  
2: C  
3: D  
4: E  

we can create these data as follows:

In [2]:
paths = pp.PathData()
paths.add_walk(torch.tensor([[0,2],[2,3]])) # A -> C -> D
paths.add_walk(torch.tensor([[0,2],[2,3]])) # A -> C -> D
paths.add_walk(torch.tensor([[1,2],[2,4]])) # B -> C -> E
paths.add_walk(torch.tensor([[1,2],[2,4]])) # B -> C -> E

In [3]:
paths.num_paths

4

In [4]:
paths.num_nodes

5

In [5]:
paths.num_edges

4

In [6]:
print(paths)

PathData with 4 paths


Paths are internally stored in a path_index dictionary, where the key is the index of the path

In [7]:
paths.paths

{0: tensor([[0, 2],
         [2, 3]]),
 1: tensor([[0, 2],
         [2, 3]]),
 2: tensor([[1, 2],
         [2, 4]]),
 3: tensor([[1, 2],
         [2, 4]])}

We can project the path_index to an edge index, which corresponds to the edge_index of all traversed edges in a graph

In [8]:
paths.edge_index

tensor([[0, 1, 2, 2],
        [2, 2, 3, 4]])

We can also create a weighted edge_index with unique edges

In [9]:
paths.edge_index_weighted

(tensor([[0, 1, 2, 2],
         [2, 2, 3, 4]]),
 tensor([2., 2., 2., 2.]))

This is a first-order graph representation. We can also generate k-th order representations with and without weights.

A k-th order edge index with m edges has the shape [2,m,k], i.e. it consists of a src and dst tensor with m entries, where each entry is a k-dim tensor representing the k nodes that constitute the higher-order node.

In [10]:
i, w = paths.edge_index_k_weighted(k=2)
print('higher-order edges =', i)
print('weights =', w)

higher-order edges = tensor([[[0, 2],
         [1, 2]],

        [[2, 3],
         [2, 4]]])
weights = tensor([2., 2.])


We can use paths to generate k-th order graphs that can be used for GNNs. To make access to nodes and edges convenient, we can pass a node_id mapping that assigns Ids to first-order nodes:

In [11]:
g = pp.HigherOrderGraph(paths, order=2, node_id=['a', 'b', 'c', 'd', 'e'])
print(g)

HigherOrderGraph (k=2) with 4 nodes and 2 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([2])

Graph attributes
	num_nodes		<class 'int'>
	node_id		<class 'list'>



Just like for a normal graph, we can iterate through nodes, which are tuples with k elements:

In [12]:
for n in g.nodes:
    print(n)

('a', 'c')
('b', 'c')
('c', 'd')
('c', 'e')


Edges are tuples with two elements, where each element is a k-th order node:

In [13]:
for e in g.edges:
    print(e)

(('a', 'c'), ('c', 'd'))
(('b', 'c'), ('c', 'e'))


The weight attribute stores tensors that capture the frequencies of edges:

In [14]:
print(g.data.is_edge_attr('edge_weight'))

True


In [15]:
g['edge_weight', ('a', 'c'), ('c', 'd')].item()

2.0

We can also add paths with frequencies:

In [16]:
paths = pp.PathData()
paths.add_walk(torch.tensor([[0,2,3],[2,3,4]]).to(pp.config['torch']['device']), 2) # A -> C -> D -> E
paths.add_walk(torch.tensor([[1,2],[2,4]]).to(pp.config['torch']['device']), 2) # B -> C -> E
paths.add_walk(torch.tensor([[0,2],[2,3]]).to(pp.config['torch']['device']), 1) # A -> C -> D

# second-order weights: 
# (0,2), (2,3) -> 3
# (1,2), (2,4) -> 2
# (0,2), (2,3) -> 1

We can read data from an n-gram file, where each line is a path consisting of comma-separated nodes:

In [17]:
p = pp.PathData.from_csv('../data/tube_paths_train.ngram')

In [18]:
print(p)

PathData with 61748 paths


In [19]:
len(p.path_freq)

61748

In [22]:
p.edge_index_k_weighted(k=2)[0].size()

torch.Size([2, 1139, 2])

In [23]:
i, w = p.edge_index_k_weighted(k=2)
torch.sum(w)

tensor(12356472.)

In [24]:
i, w = p.edge_index_k_weighted(k=3)
torch.sum(w)

tensor(10308563.)

In [25]:
i, w = p.edge_index_k_weighted(k=5)
torch.sum(w)

tensor(6791043.)

When reading paths from a csv file, the PathStorage automatically assigns node_names and path frequencies:

In [26]:
print(p.node_id)

['Southwark', 'Waterloo', 'Liverpool Street', 'Bank / Monument', 'Barking', 'West Ham', 'Tufnell Park', 'Kentish Town', 'Ruislip Gardens', 'South Ruislip', 'Turnpike Lane', 'Manor House', 'Seven Sisters', 'Finsbury Park', 'Tower Hill', 'Upminster', 'Embankment', 'Temple', 'Finchley Road', 'Willesden Green', 'Angel', 'Old Street', 'Holland Park', 'Notting Hill Gate', 'Baker Street', 'Bond Street', 'Latimer Road', 'Ladbroke Grove', 'North Harrow', 'Pinner', 'Chesham', 'Chalfont & Latimer', 'Westminster', 'Turnham Green', 'Stamford Brook', 'Belsize Park', 'Chalk Farm', 'Bow Road', 'BromleyByBow', 'Woodford', 'Buckhurst Hill', 'Hanger Lane', 'North Acton', 'Vauxhall', 'Stockwell', "Shepherd's Bush (Cen)", 'White City', 'Stratford', 'Whitechapel', 'South Woodford', 'Snaresbrook', 'East Acton', 'Moorgate', 'Edgware Road (Cir)', 'Paddington', 'Caledonian Road', 'Holloway Road', 'West Brompton', "Earl's Court", 'Queensbury', 'Kingsbury', 'Loughton', 'Debden', 'Lancaster Gate', 'Marble Arch', '

In [27]:
print(p.path_freq)

{0: 212, 1: 1271, 2: 283, 3: 103, 4: 8, 5: 339, 6: 336, 7: 80, 8: 280, 9: 61, 10: 188, 11: 419, 12: 7, 13: 218, 14: 61, 15: 12, 16: 37, 17: 544, 18: 1, 19: 43, 20: 32, 21: 29, 22: 14, 23: 234, 24: 86, 25: 650, 26: 1, 27: 14, 28: 68, 29: 19, 30: 226, 31: 936, 32: 10, 33: 30, 34: 114, 35: 1, 36: 162, 37: 39, 38: 935, 39: 117, 40: 27, 41: 90, 42: 72, 43: 20, 44: 24, 45: 35, 46: 102, 47: 33, 48: 111, 49: 15, 50: 78, 51: 18, 52: 450, 53: 143, 54: 136, 55: 4, 56: 210, 57: 2, 58: 323, 59: 105, 60: 237, 61: 1023, 62: 29, 63: 18, 64: 67, 65: 18, 66: 273, 67: 44, 68: 1, 69: 197, 70: 5, 71: 59, 72: 150, 73: 7, 74: 114, 75: 2, 76: 29, 77: 295, 78: 599, 79: 114, 80: 170, 81: 3, 82: 14, 83: 501, 84: 108, 85: 26, 86: 84, 87: 166, 88: 157, 89: 51, 90: 435, 91: 104, 92: 164, 93: 525, 94: 50, 95: 1387, 96: 114, 97: 14, 98: 27, 99: 39, 100: 195, 101: 259, 102: 141, 103: 9, 104: 1536, 105: 43, 106: 107, 107: 13, 108: 278, 109: 8, 110: 19, 111: 185, 112: 129, 113: 7, 114: 28, 115: 44, 116: 18, 117: 23, 118

The node names can be used as node_ids in a HigherOrder Graph

In [29]:
g = pp.HigherOrderGraph(p, order=2, node_id=p.node_id)
print(g)

HigherOrderGraph (k=2) with 642 nodes and 1139 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([1139])

Graph attributes
	num_nodes		<class 'int'>
	node_id		<class 'list'>



In [30]:
for e in g.edges:
    print(e, g['edge_weight', e[0], e[1]].item())

(('Southwark', 'Waterloo'), ('Waterloo', 'Embankment')) 8405.0
(('Southwark', 'Waterloo'), ('Waterloo', 'Westminster')) 91710.0
(('Southwark', 'Waterloo'), ('Waterloo', 'Lambeth North')) 204.0
(('Southwark', 'Waterloo'), ('Waterloo', 'Kennington')) 13182.0
(('Southwark', 'London Bridge'), ('London Bridge', 'Bank / Monument')) 73218.0
(('Southwark', 'London Bridge'), ('London Bridge', 'Bermondsey')) 51530.0
(('Southwark', 'London Bridge'), ('London Bridge', 'Borough')) 296.0
(('Waterloo', 'Southwark'), ('Southwark', 'London Bridge')) 140333.0
(('Waterloo', 'Embankment'), ('Embankment', 'Temple')) 1499.0
(('Waterloo', 'Embankment'), ('Embankment', 'Charing Cross')) 11663.0
(('Waterloo', 'Westminster'), ('Westminster', 'Green Park')) 103374.0
(('Waterloo', 'Westminster'), ('Westminster', "St. James's Park")) 19691.0
(('Waterloo', 'Lambeth North'), ('Lambeth North', 'Elephant & Castle')) 3719.0
(('Waterloo', 'Kennington'), ('Kennington', 'Elephant & Castle')) 3771.0
(('Waterloo', 'Kenningt

In [31]:
g = pp.HigherOrderGraph(p, order=3, node_id=p.node_id)
print(g)

HigherOrderGraph (k=3) with 1134 nodes and 1869 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([1869])

Graph attributes
	num_nodes		<class 'int'>
	node_id		<class 'list'>



In [32]:
p.edge_index_k_weighted(k=3)[0].size()

torch.Size([2, 1869, 3])

In [33]:
for e in g.edges:
    print(e, g['edge_weight', e[0], e[1]].item())

(('Southwark', 'Waterloo', 'Embankment'), ('Waterloo', 'Embankment', 'Temple')) 322.0
(('Southwark', 'Waterloo', 'Embankment'), ('Waterloo', 'Embankment', 'Charing Cross')) 3771.0
(('Southwark', 'Waterloo', 'Westminster'), ('Waterloo', 'Westminster', 'Green Park')) 67293.0
(('Southwark', 'Waterloo', 'Westminster'), ('Waterloo', 'Westminster', "St. James's Park")) 18359.0
(('Southwark', 'Waterloo', 'Lambeth North'), ('Waterloo', 'Lambeth North', 'Elephant & Castle')) 3.0
(('Southwark', 'Waterloo', 'Kennington'), ('Waterloo', 'Kennington', 'Elephant & Castle')) 3.0
(('Southwark', 'Waterloo', 'Kennington'), ('Waterloo', 'Kennington', 'Oval')) 12694.0
(('Southwark', 'London Bridge', 'Bank / Monument'), ('London Bridge', 'Bank / Monument', 'Liverpool Street')) 33691.0
(('Southwark', 'London Bridge', 'Bank / Monument'), ('London Bridge', 'Bank / Monument', 'Tower Hill')) 14582.0
(('Southwark', 'London Bridge', 'Bank / Monument'), ('London Bridge', 'Bank / Monument', 'Moorgate')) 6019.0
(('So